In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("data.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3111,3112,3113,3114,3115,3116,3117,3118,3119,3120
0,750.271,750.094,750.380,750.726,750.890,750.860,749.192,748.599,749.077,749.605,...,0.132028,0.171277,0.167796,0.141710,0.154595,0.182266,0.167652,0.160940,0.130541,0
1,750.094,750.380,750.726,750.890,750.860,749.192,748.599,749.077,749.605,749.638,...,0.171277,0.167796,0.141710,0.154595,0.182266,0.167652,0.160940,0.130541,0.216049,0
2,750.380,750.726,750.890,750.860,749.192,748.599,749.077,749.605,749.638,750.009,...,0.167796,0.141710,0.154595,0.182266,0.167652,0.160940,0.130541,0.216049,0.173276,0
3,750.726,750.890,750.860,749.192,748.599,749.077,749.605,749.638,750.009,752.381,...,0.141710,0.154595,0.182266,0.167652,0.160940,0.130541,0.216049,0.173276,0.133951,0
4,750.890,750.860,749.192,748.599,749.077,749.605,749.638,750.009,752.381,754.066,...,0.154595,0.182266,0.167652,0.160940,0.130541,0.216049,0.173276,0.133951,0.180807,1


In [ ]:
#alot of attritubes very little instances need to reduce - run baseline to get performance 
